# Fixed Effects Regression Analysis
## Coding Quiz 2

**Task**: Do a regression to estimate the fixed effect of each group. We assume that there is one single linear coefficient for all the data, plus the fixed effect of each group.

**Dataset**: homework_2.1.csv
- **time**: Treatment variable (continuous)
- **G1, G2, G3**: Outcomes for three different groups

**Model**: For each group i, we estimate:
$$Y_i = \beta_0 + \beta_1 \cdot \text{time} + \alpha_i + \epsilon$$

Where:
- $\beta_1$ is the common slope (effect of time) across all groups
- $\alpha_i$ is the fixed effect for group i (intercept adjustment)

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Set display options
pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")

## 1. Load and Explore the Data

In [2]:
# Load the data
df = pd.read_csv('homework_2.1.csv')

print("Dataset shape:", df.shape)
print("\nFirst 10 rows:")
print(df.head(10))
print("\nBasic statistics:")
print(df.describe())
print("\nData types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())

Dataset shape: (100, 4)

First 10 rows:
   time        G1        G2        G3
0     0  0.882026  1.441575  0.065409
1     1  0.210079 -0.163880  0.140310
2     2  0.509369 -0.115242  0.819830
3     3  1.150447  1.014698  0.607632
4     4  0.973779 -0.046562  0.610066
5     5 -0.438639  1.521811 -0.508478
6     6  0.535044  0.353191  0.297837
7     7 -0.005679  0.196273 -0.049015
8     8  0.028391  1.541471  0.469962
9     9  0.295299  1.330257  0.290925

Basic statistics:
             time          G1          G2          G3
count  100.000000  100.000000  100.000000  100.000000
mean    49.500000    0.524904    1.036006    0.715384
std     29.011492    0.561614    0.552593    0.581227
min      0.000000   -1.076495   -0.163880   -0.508478
25%     24.750000    0.186046    0.621088    0.298386
50%     49.500000    0.464842    1.006778    0.643041
75%     74.250000    0.947191    1.427074    1.125461
max     99.000000    1.862935    2.561618    2.321958

Data types:
time      int64
G1      

## 2. Reshape Data for Fixed Effects Analysis

We need to convert the data from wide format (G1, G2, G3 as separate columns) to long format where we have:
- One column for the outcome (Y)
- One column identifying the group
- The time variable

In [ ]:
# Reshape from wide to long format
df_long = pd.melt(df, id_vars=['time'], value_vars=['G1', 'G2', 'G3'],
                  var_name='group', value_name='outcome')

print("Reshaped data (long format):")
print(df_long.head(15))
print(f"\nTotal observations: {len(df_long)}")
print(f"Observations per group: {df_long.groupby('group').size()}")

## 3. Visualize the Data

In [ ]:
# Plot outcomes over time for each group
plt.figure(figsize=(12, 6))

for group in ['G1', 'G2', 'G3']:
    plt.plot(df['time'], df[group], 'o-', label=group, alpha=0.7, linewidth=2)

plt.xlabel('Time', fontsize=12)
plt.ylabel('Outcome', fontsize=12)
plt.title('Outcomes Over Time for Each Group', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nObservation: Each group has a different baseline level but may share")
print("a common time trend. Fixed effects will capture these group differences.")

## 4. Fixed Effects Regression Using Statsmodels

We'll use the formula API to estimate:
$$\text{outcome} = \beta_1 \cdot \text{time} + \alpha_{\text{G1}} + \alpha_{\text{G2}} + \alpha_{\text{G3}} + \epsilon$$

The `C(group)` notation creates dummy variables for each group (fixed effects).

In [ ]:
# Fixed effects regression with statsmodels
# Model: outcome ~ time + C(group)
# C(group) creates dummy variables for fixed effects

model = ols('outcome ~ time + C(group)', data=df_long).fit()

print("=" * 70)
print("FIXED EFFECTS REGRESSION RESULTS")
print("=" * 70)
print(model.summary())
print("\n" + "=" * 70)

## 5. Extract and Interpret the Fixed Effects

In [ ]:
# Extract coefficients
coefficients = model.params
time_effect = coefficients['time']

print("=" * 70)
print("EXTRACTED COEFFICIENTS")
print("=" * 70)
print(f"\nCommon Time Effect (β₁): {time_effect:.6f}")
print("\nFixed Effects (Group-specific intercepts):")
print("-" * 70)

# The baseline group (G1) is absorbed in the intercept
# Other groups show their difference from G1
intercept = coefficients['Intercept']
print(f"Group G1 fixed effect (α₁): {intercept:.6f}")

# Extract fixed effects for G2 and G3 relative to G1
if 'C(group)[T.G2]' in coefficients:
    g2_effect = intercept + coefficients['C(group)[T.G2]']
    print(f"Group G2 fixed effect (α₂): {g2_effect:.6f}")
    
if 'C(group)[T.G3]' in coefficients:
    g3_effect = intercept + coefficients['C(group)[T.G3]']
    print(f"Group G3 fixed effect (α₃): {g3_effect:.6f}")

print("\n" + "=" * 70)
print("INTERPRETATION")
print("=" * 70)
print(f"\n• The effect of time is {time_effect:.6f} units per time period")
print("  (This is the SAME for all groups)")
print("\n• Fixed effects represent baseline differences between groups:")
print("  - They capture time-invariant group characteristics")
print("  - Each group has its own intercept")
print("\n• By including fixed effects, we control for unobserved differences")
print("  between groups and isolate the true effect of time.")

## 6. Visualize Fixed Effects Regression Results

In [ ]:
# Plot actual data with fitted lines
plt.figure(figsize=(14, 6))

# Subplot 1: Actual data with fitted lines
plt.subplot(1, 2, 1)
colors = {'G1': 'blue', 'G2': 'red', 'G3': 'green'}

for group in ['G1', 'G2', 'G3']:
    group_data = df_long[df_long['group'] == group]
    plt.scatter(group_data['time'], group_data['outcome'], 
               alpha=0.5, label=f'{group} (actual)', color=colors[group])
    
    # Plot fitted line for each group
    time_range = np.linspace(df_long['time'].min(), df_long['time'].max(), 100)
    if group == 'G1':
        fitted = intercept + time_effect * time_range
    elif group == 'G2':
        fitted = g2_effect + time_effect * time_range
    else:  # G3
        fitted = g3_effect + time_effect * time_range
    
    plt.plot(time_range, fitted, '--', linewidth=2.5, 
            label=f'{group} (fitted)', color=colors[group])

plt.xlabel('Time', fontsize=12)
plt.ylabel('Outcome', fontsize=12)
plt.title('Fixed Effects Regression:\nActual vs Fitted Values', fontsize=14)
plt.legend(fontsize=9, loc='best')
plt.grid(True, alpha=0.3)

# Subplot 2: Fixed effects comparison
plt.subplot(1, 2, 2)
groups = ['G1', 'G2', 'G3']
fixed_effects = [intercept, g2_effect, g3_effect]
colors_bar = ['blue', 'red', 'green']

plt.bar(groups, fixed_effects, color=colors_bar, alpha=0.7, edgecolor='black')
plt.xlabel('Group', fontsize=12)
plt.ylabel('Fixed Effect (Intercept)', fontsize=12)
plt.title('Group-Specific Fixed Effects\n(Baseline Differences)', fontsize=14)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (group, effect) in enumerate(zip(groups, fixed_effects)):
    plt.text(i, effect + 0.02, f'{effect:.3f}', 
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nNote: All three groups have the SAME slope ({time_effect:.6f})")
print("but DIFFERENT intercepts (fixed effects).")

## 7. Alternative Method: Manual Calculation with Dummy Variables

In [ ]:
# Create dummy variables manually
df_long['G1_dummy'] = (df_long['group'] == 'G1').astype(int)
df_long['G2_dummy'] = (df_long['group'] == 'G2').astype(int)
df_long['G3_dummy'] = (df_long['group'] == 'G3').astype(int)

# Use sklearn LinearRegression (drop one dummy to avoid multicollinearity)
from sklearn.linear_model import LinearRegression

X_manual = df_long[['time', 'G2_dummy', 'G3_dummy']]
y_manual = df_long['outcome']

manual_model = LinearRegression(fit_intercept=True)
manual_model.fit(X_manual, y_manual)

print("=" * 70)
print("MANUAL CALCULATION USING SKLEARN")
print("=" * 70)
print(f"\nIntercept (G1 fixed effect): {manual_model.intercept_:.6f}")
print(f"Coefficient for time: {manual_model.coef_[0]:.6f}")
print(f"Coefficient for G2 dummy: {manual_model.coef_[1]:.6f}")
print(f"Coefficient for G3 dummy: {manual_model.coef_[2]:.6f}")

print(f"\nFixed effects:")
print(f"  G1: {manual_model.intercept_:.6f}")
print(f"  G2: {manual_model.intercept_ + manual_model.coef_[1]:.6f}")
print(f"  G3: {manual_model.intercept_ + manual_model.coef_[2]:.6f}")

print("\n✓ These match the statsmodels results!")

## 8. Summary and Key Takeaways

In [ ]:
print("=" * 70)
print("FIXED EFFECTS REGRESSION - SUMMARY")
print("=" * 70)

print(f"\n📊 MODEL: outcome = β₁·time + αᵢ + ε")
print(f"\nwhere:")
print(f"  • β₁ = {time_effect:.6f} (common time effect for all groups)")
print(f"  • α₁ = {intercept:.6f} (fixed effect for Group 1)")
print(f"  • α₂ = {g2_effect:.6f} (fixed effect for Group 2)")
print(f"  • α₃ = {g3_effect:.6f} (fixed effect for Group 3)")

print("\n" + "=" * 70)
print("KEY CONCEPTS")
print("=" * 70)

print("""
1. FIXED EFFECTS capture time-invariant differences between groups
   - Each group has its own baseline (intercept)
   - Controls for unobserved heterogeneity

2. COMMON SLOPE assumption
   - All groups experience the same effect of time
   - Only the starting point (intercept) differs

3. WHY USE FIXED EFFECTS?
   - Removes bias from group-specific characteristics
   - Focuses on within-group changes over time
   - Isolates the causal effect of the treatment (time)

4. REAL-WORLD APPLICATIONS
   - Panel data analysis (individuals/firms over time)
   - Difference-in-differences estimation
   - Controlling for unmeasured confounders
""")

print("=" * 70)
print(f"✓ R-squared: {model.rsquared:.4f}")
print(f"✓ Adjusted R-squared: {model.rsquared_adj:.4f}")
print(f"✓ Number of observations: {len(df_long)}")
print("=" * 70)

# Bootstrap Simulation Analysis
## Questions 3-5: Exploring Different Possibilities

**Task**: Given a data set, create a bootstrap simulation to try different possibilities.

**Dataset**: homework_2.2.csv
- **X**: Binary treatment variable (0 or 1)
- **Y**: Continuous outcome variable
- **Z**: Another continuous variable

**Goal**: Use bootstrap resampling to:
1. Estimate statistics with confidence intervals
2. Test different hypotheses about the data
3. Understand sampling variability

## 9. Load Bootstrap Dataset

In [ ]:
# Load the bootstrap dataset
df_boot = pd.read_csv('homework_2.2.csv')

print("Bootstrap Dataset Information:")
print("=" * 70)
print(f"Shape: {df_boot.shape}")
print(f"\nFirst 10 rows:")
print(df_boot.head(10))
print(f"\nBasic statistics:")
print(df_boot.describe())
print(f"\nValue counts for X (treatment):")
print(df_boot['X'].value_counts().sort_index())
print(f"\nCorrelations:")
print(df_boot.corr())

## 10. Bootstrap Function: Resampling with Replacement

In [ ]:
def bootstrap_sample(data, n_bootstrap=1000, statistic_func=np.mean, random_seed=42):
    """
    Perform bootstrap resampling and calculate a statistic.
    
    Parameters:
    -----------
    data : array-like
        The original data to bootstrap
    n_bootstrap : int
        Number of bootstrap samples to generate
    statistic_func : function
        Function to calculate the statistic (e.g., np.mean, np.median)
    random_seed : int
        Random seed for reproducibility
    
    Returns:
    --------
    bootstrap_stats : array
        Array of statistics from each bootstrap sample
    """
    np.random.seed(random_seed)
    n = len(data)
    bootstrap_stats = []
    
    for i in range(n_bootstrap):
        # Resample with replacement
        sample = np.random.choice(data, size=n, replace=True)
        # Calculate statistic
        stat = statistic_func(sample)
        bootstrap_stats.append(stat)
    
    return np.array(bootstrap_stats)

print("✓ Bootstrap function defined")
print("\nThis function will:")
print("  1. Create n_bootstrap samples by randomly selecting observations with replacement")
print("  2. Calculate a statistic (mean, median, etc.) for each sample")
print("  3. Return all statistics to form a bootstrap distribution")

## 11. Bootstrap Simulation: Treatment Effect Estimation

We'll estimate the Average Treatment Effect (ATE) using bootstrap to understand uncertainty.

In [ ]:
# Bootstrap for treatment effect
np.random.seed(42)
n_bootstrap = 10000

# Function to calculate treatment effect
def calculate_ate(data):
    """Calculate Average Treatment Effect"""
    treated = data[data['X'] == 1]['Y'].mean()
    control = data[data['X'] == 0]['Y'].mean()
    return treated - control

# Observed treatment effect
observed_ate = calculate_ate(df_boot)

# Bootstrap distribution of ATE
bootstrap_ates = []
for i in range(n_bootstrap):
    # Resample entire dataset with replacement
    boot_sample = df_boot.sample(n=len(df_boot), replace=True)
    ate = calculate_ate(boot_sample)
    bootstrap_ates.append(ate)

bootstrap_ates = np.array(bootstrap_ates)

# Calculate confidence intervals
ci_95 = np.percentile(bootstrap_ates, [2.5, 97.5])
ci_99 = np.percentile(bootstrap_ates, [0.5, 99.5])

print("=" * 70)
print("BOOTSTRAP TREATMENT EFFECT ANALYSIS")
print("=" * 70)
print(f"\nObserved ATE: {observed_ate:.6f}")
print(f"\nBootstrap Results ({n_bootstrap} iterations):")
print(f"  Mean of bootstrap ATEs: {bootstrap_ates.mean():.6f}")
print(f"  Standard Error: {bootstrap_ates.std():.6f}")
print(f"  95% Confidence Interval: [{ci_95[0]:.6f}, {ci_95[1]:.6f}]")
print(f"  99% Confidence Interval: [{ci_99[0]:.6f}, {ci_99[1]:.6f}]")
print("\n" + "=" * 70)

## 12. Visualize Bootstrap Distribution

In [ ]:
# Plot bootstrap distribution
plt.figure(figsize=(14, 5))

# Subplot 1: Histogram of bootstrap ATEs
plt.subplot(1, 2, 1)
plt.hist(bootstrap_ates, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
plt.axvline(observed_ate, color='red', linestyle='--', linewidth=2, label=f'Observed ATE: {observed_ate:.4f}')
plt.axvline(ci_95[0], color='green', linestyle=':', linewidth=2, label=f'95% CI')
plt.axvline(ci_95[1], color='green', linestyle=':', linewidth=2)
plt.xlabel('Average Treatment Effect', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title(f'Bootstrap Distribution of ATE\n({n_bootstrap} iterations)', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# Subplot 2: Q-Q plot to check normality
plt.subplot(1, 2, 2)
from scipy import stats
stats.probplot(bootstrap_ates, dist="norm", plot=plt)
plt.title('Q-Q Plot: Bootstrap ATE Distribution', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("  • The histogram shows the distribution of treatment effects across bootstrap samples")
print("  • The 95% CI tells us the range where the true ATE likely falls")
print("  • If 0 is NOT in the CI, the treatment effect is statistically significant")

## 13. Bootstrap for Different Statistics

Let's explore different statistics beyond the mean (median, variance, correlation, etc.)

In [ ]:
# Bootstrap different statistics for Y variable
n_boot = 5000
np.random.seed(42)

statistics = {
    'Mean': [],
    'Median': [],
    'Std Dev': [],
    '95th Percentile': []
}

for i in range(n_boot):
    boot_sample = df_boot['Y'].sample(n=len(df_boot['Y']), replace=True)
    statistics['Mean'].append(boot_sample.mean())
    statistics['Median'].append(boot_sample.median())
    statistics['Std Dev'].append(boot_sample.std())
    statistics['95th Percentile'].append(boot_sample.quantile(0.95))

# Calculate confidence intervals for each statistic
print("=" * 70)
print("BOOTSTRAP RESULTS FOR DIFFERENT STATISTICS (Variable Y)")
print("=" * 70)

for stat_name, values in statistics.items():
    observed = {
        'Mean': df_boot['Y'].mean(),
        'Median': df_boot['Y'].median(),
        'Std Dev': df_boot['Y'].std(),
        '95th Percentile': df_boot['Y'].quantile(0.95)
    }[stat_name]
    
    ci = np.percentile(values, [2.5, 97.5])
    
    print(f"\n{stat_name}:")
    print(f"  Observed: {observed:.6f}")
    print(f"  Bootstrap Mean: {np.mean(values):.6f}")
    print(f"  Bootstrap SE: {np.std(values):.6f}")
    print(f"  95% CI: [{ci[0]:.6f}, {ci[1]:.6f}]")

print("\n" + "=" * 70)

## 14. Bootstrap for Correlation Between Y and Z

In [ ]:
# Bootstrap correlation coefficient
n_boot = 5000
np.random.seed(42)

bootstrap_corrs = []
for i in range(n_boot):
    # Resample rows (keeping Y and Z pairs together)
    indices = np.random.choice(len(df_boot), size=len(df_boot), replace=True)
    boot_sample = df_boot.iloc[indices]
    corr = boot_sample['Y'].corr(boot_sample['Z'])
    bootstrap_corrs.append(corr)

bootstrap_corrs = np.array(bootstrap_corrs)

# Observed correlation
observed_corr = df_boot['Y'].corr(df_boot['Z'])
ci_corr = np.percentile(bootstrap_corrs, [2.5, 97.5])

print("=" * 70)
print("BOOTSTRAP CORRELATION ANALYSIS: Y vs Z")
print("=" * 70)
print(f"\nObserved Correlation: {observed_corr:.6f}")
print(f"Bootstrap Mean: {bootstrap_corrs.mean():.6f}")
print(f"Bootstrap SE: {bootstrap_corrs.std():.6f}")
print(f"95% CI: [{ci_corr[0]:.6f}, {ci_corr[1]:.6f}]")
print("\n" + "=" * 70)

# Plot
plt.figure(figsize=(10, 5))
plt.hist(bootstrap_corrs, bins=50, alpha=0.7, color='coral', edgecolor='black')
plt.axvline(observed_corr, color='red', linestyle='--', linewidth=2, 
           label=f'Observed: {observed_corr:.4f}')
plt.axvline(ci_corr[0], color='green', linestyle=':', linewidth=2, label='95% CI')
plt.axvline(ci_corr[1], color='green', linestyle=':', linewidth=2)
plt.axvline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Bootstrap Distribution of Correlation (Y vs Z)', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
if ci_corr[0] > 0:
    print("  ✓ Positive correlation is statistically significant (CI doesn't include 0)")
elif ci_corr[1] < 0:
    print("  ✓ Negative correlation is statistically significant (CI doesn't include 0)")
else:
    print("  ✗ Correlation is NOT statistically significant (CI includes 0)")

## 15. Bootstrap Hypothesis Testing

Test the null hypothesis that the treatment effect is zero: H₀: ATE = 0

In [ ]:
# Hypothesis test: Is ATE significantly different from 0?
print("=" * 70)
print("BOOTSTRAP HYPOTHESIS TEST")
print("=" * 70)
print(f"\nH₀: Average Treatment Effect = 0")
print(f"Hₐ: Average Treatment Effect ≠ 0")

# Two-tailed p-value
# Count how many bootstrap samples have ATE as extreme as observed
p_value = np.sum(np.abs(bootstrap_ates) >= np.abs(observed_ate)) / n_bootstrap

print(f"\nObserved ATE: {observed_ate:.6f}")
print(f"Bootstrap p-value: {p_value:.4f}")

if p_value < 0.01:
    print(f"\n✓✓ HIGHLY SIGNIFICANT: Reject H₀ at α=0.01")
    print(f"   The treatment has a statistically significant effect.")
elif p_value < 0.05:
    print(f"\n✓ SIGNIFICANT: Reject H₀ at α=0.05")
    print(f"   The treatment has a statistically significant effect.")
else:
    print(f"\n✗ NOT SIGNIFICANT: Fail to reject H₀")
    print(f"   Insufficient evidence that treatment has an effect.")

# Alternative: Check if 0 is in confidence interval
ate_in_ci = (ci_95[0] <= 0 <= ci_95[1])
print(f"\n95% CI contains 0: {ate_in_ci}")
if not ate_in_ci:
    print("   → This confirms statistical significance")

print("\n" + "=" * 70)

## 16. Bootstrap Summary: Key Concepts and Applications

In [ ]:
print("=" * 70)
print("BOOTSTRAP METHOD - COMPREHENSIVE SUMMARY")
print("=" * 70)

print("""
📚 WHAT IS BOOTSTRAP?
   Bootstrap is a resampling method that estimates the sampling distribution
   of a statistic by repeatedly drawing samples WITH REPLACEMENT from the data.

🔄 HOW IT WORKS:
   1. Draw a random sample of size n from the original data (with replacement)
   2. Calculate the statistic of interest (mean, median, correlation, etc.)
   3. Repeat steps 1-2 many times (typically 1,000-10,000)
   4. The distribution of these statistics approximates the sampling distribution

✨ ADVANTAGES:
   • No assumptions about the underlying distribution (non-parametric)
   • Works for complex statistics where formulas don't exist
   • Provides confidence intervals and standard errors
   • Useful for small samples

⚠️ LIMITATIONS:
   • Computationally intensive
   • Assumes sample is representative of population
   • May not work well for extreme values (e.g., max, min)

📊 APPLICATIONS IN THIS NOTEBOOK:
""")

results_summary = {
    "Treatment Effect (ATE)": {
        "Estimate": f"{observed_ate:.4f}",
        "95% CI": f"[{ci_95[0]:.4f}, {ci_95[1]:.4f}]",
        "SE": f"{bootstrap_ates.std():.4f}"
    },
    "Correlation (Y vs Z)": {
        "Estimate": f"{observed_corr:.4f}",
        "95% CI": f"[{ci_corr[0]:.4f}, {ci_corr[1]:.4f}]",
        "SE": f"{bootstrap_corrs.std():.4f}"
    }
}

for analysis, metrics in results_summary.items():
    print(f"\n   • {analysis}:")
    for metric, value in metrics.items():
        print(f"     - {metric}: {value}")

print("\n" + "=" * 70)
print("🎯 KEY TAKEAWAY")
print("=" * 70)
print("""
Bootstrap allows us to quantify uncertainty in our estimates without making
strong distributional assumptions. It's particularly valuable when:
  • Theoretical formulas for standard errors are unavailable
  • The sample size is small or moderate
  • The sampling distribution is unknown or non-normal
  • We need confidence intervals for complex statistics
""")

In [4]:
import numpy as np

def bootstrap_pareto_variance(sample_size, n_bootstrap=1000, shape=2.5):
    original_sample = np.random.pareto(shape, sample_size)
    bootstrap_means = []
    
    for _ in range(n_bootstrap):
        bootstrap_sample = np.random.choice(original_sample, size=sample_size, replace=True)
        bootstrap_means.append(np.mean(bootstrap_sample))
    
    return np.var(bootstrap_means)

sample_sizes = [50, 100, 500, 1000, 5000]
for size in sample_sizes:
    var = bootstrap_pareto_variance(size)
    print(f"Sample size: {size}, Variance: {var:.4f}")

Sample size: 50, Variance: 0.0163
Sample size: 100, Variance: 0.0116
Sample size: 500, Variance: 0.0017
Sample size: 1000, Variance: 0.0015
Sample size: 5000, Variance: 0.0004
